In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [20]:
data = pd.read_csv('cleaned.csv')
data.head(7)

,Unnamed: 0,ListingNumber,ListingCreationDate,Term,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,...,CreditGrade,LoanStatus,BorrowerState,Occupation,EmploymentStatus,IsBorrowerHomeowner,CurrentlyInGroup,IncomeRange,IncomeVerifiable,LoanOriginationQuarter
0,0,193129,2007-08-26 19:09:29.263,36,1,0.16516,0.1580,0.1380,0.16150,0.0724,...,4,Completed,CO,Other,Self-employed,1,1,"$25,000-49,999",1,Q3 2007
1,1,1209647,2014-02-27 08:28:07.900,36,0,0.12016,0.0920,0.0820,0.07960,0.0249,...,0,Current,CO,Professional,Employed,0,0,"$50,000-74,999",1,Q1 2014
2,2,658116,2012-10-22 11:02:35.010,36,0,0.12528,0.0974,0.0874,0.08490,0.0249,...,0,Current,GA,Skilled Labor,Employed,1,0,"$25,000-49,999",1,Q4 2012
3,3,909464,2013-09-14 18:38:39.097,36,0,0.24614,0.2085,0.1985,0.18316,0.0925,...,0,Current,MN,Executive,Employed,1,0,"$100,000+",1,Q3 2013
4,4,1074836,2013-12-14 08:26:37.093,60,0,0.15425,0.1314,0.1214,0.11567,0.0449,...,0,Current,NM,Professional,Employed,1,0,"$100,000+",1,Q4 2013
5,5,750899,2013-04-12 09:52:56.147,36,0,0.31032,0.2712,0.2612,0.23820,0.1275,...,0,Current,KS,Sales - Retail,Employed,0,0,"$25,000-49,999",1,Q2 2013
6,6,768193,2013-05-05 06:49:27.493,36,0,0.23939,0.2019,0.1919,0.17830,0.0799,...,0,Current,CA,Laborer,Employed,0,0,"$25,000-49,999",1,Q2 2013


In [21]:
y = data['LoanStatus']
X = data.drop('LoanStatus',axis=1)

In [22]:
y

0                      Completed
1                        Current
2                        Current
3                        Current
4                        Current
                   ...          
103355                   Current
103356    FinalPaymentInProgress
103357                   Current
103358                 Completed
103359                   Current
Name: LoanStatus, Length: 103360, dtype: object

In [23]:
# y = pd.get_dummies(y)


In [24]:
y

0                      Completed
1                        Current
2                        Current
3                        Current
4                        Current
                   ...          
103355                   Current
103356    FinalPaymentInProgress
103357                   Current
103358                 Completed
103359                   Current
Name: LoanStatus, Length: 103360, dtype: object

In [25]:
cat_X = X.select_dtypes(include=('object'))
cat_X

,ListingCreationDate,DateCreditPulled,FirstRecordedCreditLine,LoanOriginationDate,BorrowerState,Occupation,EmploymentStatus,IncomeRange,LoanOriginationQuarter
0,2007-08-26 19:09:29.263,2007-08-26 18:41:46.780,2001-10-11 00:00:00.000,2007-09-12 00:00:00.000,CO,Other,Self-employed,"$25,000-49,999",Q3 2007
1,2014-02-27 08:28:07.900,2014-02-27 08:28:14.000,1996-03-18 00:00:00.000,2014-03-03 00:00:00.000,CO,Professional,Employed,"$50,000-74,999",Q1 2014
2,2012-10-22 11:02:35.010,2012-10-22 11:02:32.000,1983-02-28 00:00:00.000,2012-11-01 00:00:00.000,GA,Skilled Labor,Employed,"$25,000-49,999",Q4 2012
3,2013-09-14 18:38:39.097,2013-09-14 18:38:44.000,2004-02-20 00:00:00.000,2013-09-20 00:00:00.000,MN,Executive,Employed,"$100,000+",Q3 2013
4,2013-12-14 08:26:37.093,2013-12-14 08:26:40.000,1973-03-01 00:00:00.000,2013-12-24 00:00:00.000,NM,Professional,Employed,"$100,000+",Q4 2013
...,...,...,...,...,...,...,...,...,...
103355,2013-04-14 05:55:02.663,2013-04-14 05:54:58.000,1997-09-01 00:00:00.000,2013-04-22 00:00:00.000,IL,Food Service Management,Employed,"$50,000-74,999",Q2 2013
103356,2011-11-03 20:42:55.333,2011-11-03 20:42:53.000,1992-01-17 00:00:00.000,2011-11-07 00:00:00.000,PA,Professional,Employed,"$75,000-99,999",Q4 2011
103357,2013-12-13 05:49:12.703,2013-12-13 05:49:15.000,2002-02-25 00:00:00.000,2013-12-23 00:00:00.000,TX,Other,Employed,"$25,000-49,999",Q4 2013
103358,2011-11-14 13:18:26.597,2011-11-14 13:18:24.000,1993-12-01 00:00:00.000,2011-11-21 00:00:00.000,GA,Food Service,Full-time,"$25,000-49,999",Q4 2011


In [26]:
X = X.drop(columns=cat_X.columns)

In [27]:
for column in cat_X.columns:
    print(f'{column} : {len(cat_X[column].unique())}')

ListingCreationDate : 102487
DateCreditPulled : 102416
FirstRecordedCreditLine : 11447
LoanOriginationDate : 1591
BorrowerState : 51
Occupation : 67
EmploymentStatus : 7
IncomeRange : 8
LoanOriginationQuarter : 28


In [28]:
cat_X = cat_X.drop(columns=['ListingCreationDate','DateCreditPulled','FirstRecordedCreditLine','LoanOriginationDate'])

In [29]:
cat_X = pd.get_dummies(cat_X,drop_first=True)

In [30]:
X.join(cat_X)

,Unnamed: 0,ListingNumber,Term,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,...,LoanOriginationQuarter_Q3 2011,LoanOriginationQuarter_Q3 2012,LoanOriginationQuarter_Q3 2013,LoanOriginationQuarter_Q4 2007,LoanOriginationQuarter_Q4 2008,LoanOriginationQuarter_Q4 2009,LoanOriginationQuarter_Q4 2010,LoanOriginationQuarter_Q4 2011,LoanOriginationQuarter_Q4 2012,LoanOriginationQuarter_Q4 2013
0,0,193129,36,1,0.16516,0.1580,0.1380,0.16150,0.0724,0.09170,...,0,0,0,0,0,0,0,0,0,0
1,1,1209647,36,0,0.12016,0.0920,0.0820,0.07960,0.0249,0.05470,...,0,0,0,0,0,0,0,0,0,0
2,2,658116,36,0,0.12528,0.0974,0.0874,0.08490,0.0249,0.06000,...,0,0,0,0,0,0,0,0,1,0
3,3,909464,36,0,0.24614,0.2085,0.1985,0.18316,0.0925,0.09066,...,0,0,1,0,0,0,0,0,0,0
4,4,1074836,60,0,0.15425,0.1314,0.1214,0.11567,0.0449,0.07077,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103355,103355,753087,36,0,0.22354,0.1864,0.1764,0.16490,0.0699,0.09500,...,0,0,0,0,0,0,0,0,0,0
103356,103356,537216,36,0,0.13220,0.1110,0.1010,0.10070,0.0200,0.08070,...,0,0,0,0,0,0,0,1,0,0
103357,103357,1069178,60,0,0.23984,0.2150,0.2050,0.18828,0.1025,0.08578,...,0,0,0,0,0,0,0,0,0,1
103358,103358,539056,60,1,0.28408,0.2605,0.2505,0.24450,0.0850,0.15950,...,0,0,0,0,0,0,0,1,0,0


In [31]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [34]:
y.shape

(103360,)

In [35]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs',penalty="l2")
# cv = RepeatedStratifiedKFold(n_splits=12, n_repeats=3, random_state=1)
# n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [36]:
history = model.fit(X_train,y_train)

C:\Users\m7mde\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
print(f'accuracy is {model.score(X_test,y_test)*100}%')

accuracy is 97.1781475748194%
